In [1]:
#!/bin/python3

import math
import os
import random
import re
import sys

#
# Complete the 'decibinaryNumbers' function below.
#
# The function is expected to return a LONG_INTEGER.
# The function accepts LONG_INTEGER x as parameter.
#

In [2]:
from heapq import heappush, heappop
from collections import defaultdict
from bisect import bisect_left, bisect_right
import math
import logging

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)


#Remove existing handlers from root logger
root_logger = logging.getLogger()
root_logger.setLevel(logging.DEBUG)
root_logger.handlers = []

#Create custom logger

logger = logging.getLogger(__name__)
ch = logging.StreamHandler(stream=sys.stdout)
ch.setLevel(logging.DEBUG)

formatter = logging.Formatter('%(message)s')
ch.setFormatter(formatter)

logger.addHandler(ch)

logger.setLevel(logging.WARNING)


def generate_start(value):
    
    if value < 10:
        return [value]
        
    else:
        rem = value
        mult = 1
        
        list_digit = []

        logger.debug('in first while')
        while rem >= mult:
            logger.debug(f'rem = {rem}')
            max_digit = min(8,rem // mult)
            
            list_digit.append(max_digit)
            rem -= max_digit*mult
            
            mult *= 2
            
        #Distribute leftovers
        logger.debug('in second while')
        while rem > 0:
            logger.debug(f'rem = {rem}')
            
            index = int(math.log(rem,2))
            list_digit[index] += 1
            rem -= 2**index
        
    return list_digit
    
def get_count(pv, mod_rem, dict_counts):
    
    if dict_counts is None:
        dict_counts = defaultdict(dict)
        dict_counts[0] = {0:[2,4,5,5,5,3,1,0,0], 1:[1,3,5,5,5,4,2,0,0]}
    
    #Accumulate
    if not (pv in dict_counts and mod_rem in dict_counts[pv]):
        list_generate = []
        
        cur_pv = pv
        cur_mod_rem = mod_rem
        
        while cur_mod_rem not in dict_counts[cur_pv]:
            
            list_generate.append((cur_pv, cur_mod_rem))
            
            if cur_mod_rem < 2**cur_pv:
                cur_mode_rem = cur_mod_rem
                
            else:
                cur_mod_rem -= 2**cur_pv
            
            cur_pv -= 1
            
        logger.debug(list_generate)
        
        #Generate
        for cur_pv, cur_mod_rem in reversed(list_generate):
            
            sum_base = 5**(cur_pv+1)
            even_case = cur_mod_rem < 2**cur_pv
            
            #Generate Even
            if even_case:
                list_input = dict_counts[cur_pv-1][cur_mod_rem]
                list_1 = []
                list_2 = [sum_base]
                for j in range(2, 9, 2):
                    sum_subset = sum(list_input[0:j])
                    list_1.append(sum_subset)
                    list_2.append(sum_base - sum_subset)
                    
#                 list_final = list_1 + list_2[:-1]
                list_final = list_1 + list_2
        
            #Generate Odd
            else:
                list_input = dict_counts[cur_pv-1][cur_mod_rem - 2**cur_pv]
                list_1 = []
                list_2 = [sum_base]
                for j in range(1,8,2):
                    sum_pair = sum(list_input[0:j])
                    list_1.append(sum_pair)
                    list_2.append(sum_base - sum_pair)
                    
                list_final = list_1 + list_2
                    
                    
            dict_counts[cur_pv][cur_mod_rem] = list_final
            
    return dict_counts, dict_counts[pv][mod_rem]
    
    
def generate_index(x, list_index):

    if list_index is None:
        list_index= [(1,2,1, 1),
                     (3,5,2, 2), 
                     (7,11,4, 4), 
                     (15,21,6, 6), 
                     (27,37,10, 10)]
    
    while x >= list_index[-1][0]:
          
        i = len(list_index)

        
        B4 = list_index[(i)//2][3]
        B2 = list_index[(i-2)//2][3]
        
        A = list_index[i-1][2] + B4
        next_count =  A - list_index[i-5][2]

        starting = list_index[-1][1] + list_index[-1][3]
         
        list_index.append((starting, starting+next_count, A, next_count))
        
    return list_index
        
            
            

In [3]:
def decibinaryNumbers(x):
    # Write your code here
    
    if x == 1:
        return 0
    elif x == 2:
        return 1
    
    
    
    global list_index, dict_counts

    # list_index not initialized
    if 'list_index' not in globals() or list_index == None:
        logger.info(f'list_index NOT in globals')
        
        list_index = generate_index(x, None)
        
#         i_index = len(list_index)-2
    
#     logger.info(f'list_index in globals')

    i_index = bisect_right(list_index, x, key = lambda tup: tup[0]) + 1
    
    if i_index < len(list_index) and x < list_index[i_index][0]:
        logger.info(f'in if')
        i_index -= 2
    # Generate
    else:
        logger.info(f'in else')
        list_index = generate_index(x, list_index)
        i_index = len(list_index)-2

    
    logger.info(f'i_index = {i_index}')
    
    
    value = i_index*2    
    
    if x >= list_index[i_index][1]:
        lower_bound = list_index[i_index][1]
        value += 1 
        
    else:
        lower_bound = list_index[i_index][0]
        
    logger.info(f'value = {value}')
        
    logger.info(f'x = {x}') 
    logger.info(f'lower_bound = {lower_bound}') 
    
    list_digits = [0]* int(math.log(value,2) + 1)
    new_digits = generate_start(value)
    list_digits[0:len(new_digits)] = new_digits
    
    logger.info(f'decib_start = {int("".join(map(str, reversed(list_digits))))}')
    
    
#     while len(list_digits) <= int(math.log(value,2)):
#         list_digits.append(0)
    
    
    rem = x - lower_bound
    
        
    logger.info(f'rem index = {rem}')


    if 'dict_counts' not in globals():
        dict_counts, _ = get_count(0,0,None)
    
    
    rem_val = value
    index = lower_bound
    
    k_break = 0
    
#     max_pv = 
    
    cur_pv = int(math.log((rem_val + 8), 2)) - 5
#     cur_pv = int(math.log((rem + 8), 2)) - 5
    cur_pv = max(0,cur_pv)
    
    while len(list_digits) > 2 and rem > 5 and cur_pv >= 0:
#     while rem >= 5:

#         cur_pv = int(math.log(rem,5)) - 1


        mod_val = 2**(cur_pv+2)
        mod_rem = (rem_val+ 8) % mod_val // 2
        
        logger.info(f'----------------------------------------')
        
        logger.info(f'cur_pv {cur_pv}')
        logger.info(f'mod_val {mod_val}')
        logger.info(f'mod_rem {mod_rem}')
        
#         break
        
        dict_counts, list_counts = get_count(cur_pv, mod_rem, dict_counts)

#         cur_digit = 0 if cur_pv >= len(list_digits)  else list_digits[cur_pv]
    
#         if cur_digit == 0:
#             is_odd = 1 if list_digits[cur_pv-1] % 2 == 1 else 0
            
#         else:
#             is_odd = 1 if cur_digit % 2 == 1 else 0
        
        i_start = 0
        i_end = len(list_counts)
#         quot_val = rem_val // 2**(cur_pv+2) + 1
        quot_val = (rem_val + 8) // 2**(cur_pv+2)
        
        logger.info(f'quot_val {quot_val}')
        
        if quot_val < 8:
            i_start += 8 - quot_val
            
        elif quot_val >= 8 and quot_val < 10:
            pass
            
        elif quot_val == 10:
            logger.info('quot_val == 10')
            if rem_val % mod_val > 4:
                logger.info('in > 4')
                i_end -= 1
            
        elif quot_val > 10 and quot_val < 18:
            i_end -= quot_val - 9
#             i_end -= quot_val - 10
        else:
            if quot_val % 2 == 0:
                i_end = 2
            else:
                i_end = 1
                 
            
        logger.info(f'i_start {i_start}')
        logger.info(f'i_end {i_end}')
        logger.info(f'list_counts {list_counts}')
        logger.info(f'list_counts actual {list_counts[i_start:i_end]}')
        logger.info(f'length of counts: {i_end - i_start}')
            
#         break
        
        cumsum = 0
        increase = 0
        for count in list_counts[i_start:i_end]:
            if cumsum + count > rem:
                break
            
            increase +=1
            cumsum += count
            
            
        logger.info(f'cumsum = {cumsum}')
        logger.info(f'increase = {increase}') 
                 
        
#         if increase == 0 or increase == i_end - i_start:
#             print('in equal')
#             break

        
        if increase == 0:
            logger.info('increase == 0')
            logger.info(f'rem_val = {rem_val}')
            logger.info(f'digit = { list_digits[cur_pv+2]}')
            
            rem_val -= 2**(cur_pv+2) * list_digits[cur_pv+2]
            cur_pv -= 1
            
            logger.info(f'rem_val == {rem_val}')
            continue
        
        elif increase == i_end - i_start:
            logger.info('increase == len counts')
            logger.info('increasing pv (first)')
            cur_pv += 1
            
#             if cur_pv > max_pv:
#                 logger.warning(f'cur_pv > max_pv: {cur_pv} > {max_pv}')
#                 break

            continue
#             while list_digits[cur_pv+2] + increase > 9:
#                 logger.info('increasing pv')
#                 cur_pv += 1
#             increase = 1

        rem -= cumsum
        index += cumsum
            
        
        list_digits[cur_pv+2] += increase
        list_digits[0:cur_pv+2] = [0] * (cur_pv+2)

        
        rem_val -= 2**(cur_pv+2) * list_digits[cur_pv+2]
         
        
        logger.info(f'rem_val = {rem_val}')
        
        
        new_digits = generate_start(rem_val)
         
        logger.info(f'new_digits = {new_digits}')
        
        list_digits[0:len(new_digits)] = new_digits
        
        cur_pv -= 1
        
        logger.info(f'-------------------')
        logger.info(f'index = {index}')
        logger.info(f'rem index = {rem}')
        logger.info(f'rem value = {rem_val}')
        logger.info(f'list_digits {list_digits}')
        logger.info(f'decibinary = {int("".join(map(str, reversed(list_digits))))}')
        
    
    #Naive implementation?:
    ##Assumes list_digits[1] + rem is always < 10
    ##Assumes list_digits[0] - 2*rem is always >= 0
#     if rem != 0:
# #         print('in rem < 5')
#         list_digits[1] += rem
#         list_digits[0] -= 2*rem
        
    cur_pv = 1
    
    logger.info('in rem < 5')
    
    while rem != 0:
        logger.info(f'in rem == {rem}')
        logger.info(f'list_digits == {list_digits}')
        
#         logger.info(f'cur_pv = {cur_pv}')
#         logger.info(f'cur_pv = {cur_pv}')
        
        
        if list_digits[cur_pv] < 9 and list_digits[cur_pv-1] >= 2:
            
            
            list_digits[cur_pv] += 1
            rem_val = sum([list_digits[i] * 2**i for i in range(cur_pv)]) - 2**cur_pv
            list_digits[0:cur_pv] = [0] * (cur_pv)
            new_digits = generate_start(rem_val)
            list_digits[0:len(new_digits)] = new_digits
            
            rem -= 1
            cur_pv = 1
            
        else:
            cur_pv +=1
                
    
    return int("".join(map(str, reversed(list_digits))))
        


# Profile

In [5]:
import cProfile

In [34]:
import random

In [7]:
def random_run():
    target_index = random.randint(1,10**16)
    return decibinaryNumbers(target_index)

In [8]:
def random_batch():
    for i in range(50000):
        random_run()

In [9]:
cProfile.run('random_run()')

         250249 function calls in 0.347 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.312    0.156    0.345    0.173 2345458327.py:125(generate_index)
       18    0.000    0.000    0.001    0.000 2345458327.py:29(generate_start)
       17    0.000    0.000    0.000    0.000 2345458327.py:61(get_count)
        1    0.001    0.001    0.347    0.347 493151430.py:1(decibinaryNumbers)
       17    0.000    0.000    0.000    0.000 493151430.py:23(<lambda>)
        4    0.000    0.000    0.000    0.000 493151430.py:247(<listcomp>)
        1    0.000    0.000    0.347    0.347 533731235.py:1(random_run)
        1    0.000    0.000    0.347    0.347 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 __init__.py:1319(disable)
      190    0.000    0.000    0.000    0.000 __init__.py:1467(debug)
      323    0.000    0.000    0.000    0.000 __init__.py:1479(info)
        2    0.000    0.000    0.000  

## Batch

In [10]:
%%time
random_batch()

CPU times: total: 32.8 s
Wall time: 32.9 s


In [42]:
logger.setLevel(logging.WARNING)

In [54]:
dict_counts = None
list_index = None
cProfile.run('random_batch()')

         21087098 function calls in 22.155 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   773523    8.863    0.000   12.259    0.000 2611218294.py:1(generate_start)
   775121    1.124    0.000    1.372    0.000 2611218294.py:33(get_count)
       10    0.309    0.031    0.341    0.034 2611218294.py:97(generate_index)
    50000    6.536    0.000   21.757    0.000 3062967072.py:1(decibinaryNumbers)
   857943    0.201    0.000    0.201    0.000 3062967072.py:22(<lambda>)
   104732    0.066    0.000    0.066    0.000 3062967072.py:246(<listcomp>)
        1    0.034    0.034   22.155   22.155 4073945833.py:1(random_batch)
    50000    0.088    0.000   22.121    0.000 533731235.py:1(random_run)
        1    0.000    0.000   22.155   22.155 <string>:1(<module>)
    50000    0.074    0.000    0.108    0.000 random.py:235(_randbelow_with_getrandbits)
    50000    0.104    0.000    0.232    0.000 random.py:284(randrange)
    50000